In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.preprocessing import MultiLabelBinarizer

In [16]:
X = pd.read_csv(
    '../dumps/models/deepwalk_BlogCatalog_d128.csv',
    delim_whitespace=True, header=None,
    skiprows=1,
    index_col=0
).sort_index()
X.index.name=None

In [3]:
y = pd.read_csv(
    '../data/BlogCatalog-dataset/data/group-edges.csv', 
    header=None, 
    delim_whitespace=True, 
    index_col=0
).sort_index()
y.index.name = None

yy = []
for i in range(1, 10313):
    values = y.loc[i].values
    if len(values) != 1:
        values = [item[0] for item in values]
    else:
        values = list(values)
    yy.append(values)
    
y_multilabel = MultiLabelBinarizer().fit_transform(yy)

In [17]:
X.loc[6858:6859]

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
6858,-0.034469,-0.038209,0.078143,0.497694,-0.406538,0.492025,-0.532285,-0.181553,-0.158531,-0.209213,...,0.108826,-0.107357,-0.587715,-0.084544,0.316130,0.229775,0.151733,0.018097,0.452437,0.132487
6859,-0.078283,-0.074830,-0.055449,0.281653,0.084093,0.263505,-0.037267,-0.084210,-0.189937,0.041722,...,0.107443,-0.253507,-0.275865,-0.023679,0.161489,0.321386,0.094676,-0.119529,0.157206,0.071817


In [18]:
y_multilabel[6857]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from src.classifiers import MultilabelOVRClassifier

from sklearn.model_selection import cross_val_score, ShuffleSplit

In [7]:
clf1 = LogisticRegression(C=100)

In [8]:
clf2 = OneVsRestClassifier(LogisticRegression(C=1, verbose=0), n_jobs=-1)

In [9]:
MultilabelOVRClassifier.set_labels(X.index.values, y_multilabel)
clf3 = MultilabelOVRClassifier(LogisticRegression(C=1, verbose=0), n_jobs=-1)

In [10]:
clf = clf3

In [11]:
clf.fit(X, y_multilabel)

MultilabelOVRClassifier(estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
            n_jobs=-1)

In [13]:
pred = clf.predict(X)

In [17]:
pred[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
len(X)

In [18]:
len([_ for arr in clf.predict(X) if sum(arr)>1])

10312

In [ ]:
[sorted(item) for item in clf.predict_proba(X.loc[1:7])]

In [12]:
ss = ShuffleSplit(n_splits=3, random_state=42, train_size=0.6)

In [19]:
scores = cross_val_score(clf, X, y_multilabel, cv=ss.split(X), scoring='f1_micro', verbose=1)
print(scores)
print(scores.mean())

[ 0.3869596   0.37990365  0.37893983]
0.381934359608


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.8s finished


In [20]:
scores = cross_val_score(clf, X, y_multilabel, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
print(scores)
print(scores.mean())

/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classi

[ 0.21662256  0.20285568  0.17924865]
0.199575629232


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    3.3s finished


In [23]:
y = pd.read_csv(
    '../data/BlogCatalog-dataset/data/group-edges.csv', 
    header=None, 
    delim_whitespace=True, 
    index_col=0
).sort_index()
y.index.name = None

yy = []
for i in range(1, 10313):
    values = y.loc[i].values
    if len(values) != 1:
        values = [item[0] for item in values]
    else:
        values = list(values)
    yy.append(values)
    
y_multilabel = MultiLabelBinarizer().fit_transform(yy)

names = [
    'deepwalk_BlogCatalog_d32',
    'deepwalk_BlogCatalog_d64',
    'deepwalk_BlogCatalog_d128',
    'n2v_BlogCatalog_p1_q1_d32',
    'n2v_BlogCatalog_p1_q1_d64',
    'n2v_BlogCatalog_p1_q1_d128',
]
results = {}
for name in names:
    results[name] = {}
    X = pd.read_csv(
        '../dumps/models/{}.csv'.format(name),
        delim_whitespace=Tru'n2v_BlogCatalog_p1_q1_d64',e, header=None,
        skiprows=1,
        index_col=0
    ).sort_index()
    X.index.name=None
    print(name)
    ss = ShuffleSplit(n_splits=10, random_state=42, train_size=0.1)
    scores = cross_val_score(clf, X, y_multilabel, cv=ss.split(X), scoring='f1_micro', verbose=1)
    print(scores)
    print(scores.mean())
    results[name]['f1_micro'] = scores.mean()

print(results)

deepwalk_BlogCatalog_d32


/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 37 is present i

[ 0.31750532  0.3083276   0.33667622  0.33032616  0.28840782  0.32535211
  0.33039946  0.29781421  0.31302521  0.32052162]
0.316835572337
deepwalk_BlogCatalog_d64


/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 37 is present i

[ 0.31041814  0.32346869  0.32234957  0.31644691  0.29120112  0.31830986
  0.33310765  0.30532787  0.30742297  0.32120796]
0.314926073743
deepwalk_BlogCatalog_d128


...

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    3.4s finished
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-pac

[ 0.35861091  0.35650379  0.36676218  0.34212353  0.32541899  0.3443662
  0.34123223  0.32035519  0.34103641  0.35552505]
0.345193447888
n2v_BlogCatalog_p1_q1_d128


/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/student_panov/MrTsepa/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 37 is present i

[ 0.33876683  0.34618032  0.36389685  0.34559334  0.32122905  0.3443662
  0.34800271  0.33333333  0.34453782  0.34934798]
0.343525441412
{'deepwalk_BlogCatalog_d32': {'f1_micro': 0.31683557233719906}, 'deepwalk_BlogCatalog_d64': {'f1_micro': 0.31492607374280857}, 'deepwalk_BlogCatalog_d128': {'f1_micro': 0.31994459243798984}, 'n2v_BlogCatalog_p1_q1_d32': {'f1_micro': 0.33877837457371374}, 'n2v_BlogCatalog_p1_q1_d64': {'f1_micro': 0.34519344788800799}, 'n2v_BlogCatalog_p1_q1_d128': {'f1_micro': 0.34352544141234664}}


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.7s finished


In [25]:
y = pd.read_csv(
    '../data/BlogCatalog-dataset/data/group-edges.csv', 
    header=None, 
    delim_whitespace=True, 
    index_col=0
).sort_index()
y.index.name = None

yy = []
for i in range(1, 10313):
    values = y.loc[i].values
    if len(values) != 1:
        values = [item[0] for item in values]
    else:
        values = list(values)
    yy.append(values)
    
y_multilabel = MultiLabelBinarizer().fit_transform(yy)

names = [
    'deepwalk_BlogCatalog_d32',
    'deepwalk_BlogCatalog_d64',
    'deepwalk_BlogCatalog_d128',
    'n2v_BlogCatalog_p1_q1_d32',
    'n2v_BlogCatalog_p1_q1_d64',
    'n2v_BlogCatalog_p1_q1_d128',
]
results = {}
for name in names:
    results[name] = {}
    X = pd.read_csv(
        '../dumps/models/{}.csv'.format(name),
        delim_whitespace=True, header=None,
        skiprows=1,
        index_col=0
    ).sort_index()
    X.index.name=None
    print(name)
    ss = ShuffleSplit(n_splits=10, random_state=42, train_size=0.6)
    scores = cross_val_score(clf, X, y_multilabel, cv=ss.split(X), scoring='f1_micro', verbose=1)
    print(scores)
    print(scores.mean())
    results[name]['f1_micro'] = scores.mean()

print(results)

deepwalk_BlogCatalog_d32


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    6.9s finished


[ 0.35223246  0.33241569  0.35959885  0.34489938  0.32472067  0.34507042
  0.34326337  0.32513661  0.33753501  0.34797529]
0.341284776257
deepwalk_BlogCatalog_d64


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   12.2s finished


[ 0.35861091  0.35856848  0.3717765   0.35600278  0.3372905   0.3528169
  0.3676371   0.33811475  0.3557423   0.35895676]
0.355551699132
deepwalk_BlogCatalog_d128
[ 0.37207654  0.37302134  0.38467049  0.36502429  0.34986034  0.36338028
  0.38659445  0.3442623   0.36344538  0.36376115]
0.36660965438
n2v_BlogCatalog_p1_q1_d32


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   24.7s finished


[ 0.36853296  0.36820372  0.37679083  0.35669674  0.33868715  0.37323944
  0.36018957  0.34699454  0.36834734  0.35415237]
0.361183464437
n2v_BlogCatalog_p1_q1_d64


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.9s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    8.7s finished


[ 0.3869596   0.37990365  0.37893983  0.36918806  0.3547486   0.37816901
  0.3859174   0.34221311  0.37254902  0.3726836 ]
0.372127189103
n2v_BlogCatalog_p1_q1_d128
[ 0.37491141  0.37783895  0.39469914  0.37612769  0.37290503  0.37746479
  0.38185511  0.36680328  0.37044818  0.37131091]
0.376436449291
{'deepwalk_BlogCatalog_d32': {'f1_micro': 0.34128477625714709}, 'deepwalk_BlogCatalog_d64': {'f1_micro': 0.3555516991322108}, 'deepwalk_BlogCatalog_d128': {'f1_micro': 0.36660965438019455}, 'n2v_BlogCatalog_p1_q1_d32': {'f1_micro': 0.36118346443725552}, 'n2v_BlogCatalog_p1_q1_d64': {'f1_micro': 0.37212718910335757}, 'n2v_BlogCatalog_p1_q1_d128': {'f1_micro': 0.3764364492914824}}


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   14.0s finished


In [27]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results)

{   'deepwalk_BlogCatalog_d128': {'f1_micro': 0.36660965438019455},
    'deepwalk_BlogCatalog_d32': {'f1_micro': 0.34128477625714709},
    'deepwalk_BlogCatalog_d64': {'f1_micro': 0.3555516991322108},
    'n2v_BlogCatalog_p1_q1_d128': {'f1_micro': 0.3764364492914824},
    'n2v_BlogCatalog_p1_q1_d32': {'f1_micro': 0.36118346443725552},
    'n2v_BlogCatalog_p1_q1_d64': {'f1_micro': 0.37212718910335757}}
